<a href="https://colab.research.google.com/github/KarthikeyanBaskaran/voice_to_form/blob/main/Healthcare_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Libraries


import pandas as pd
import numpy as np


In [2]:
orginal = pd.read_csv('https://raw.githubusercontent.com/KarthikeyanBaskaran/voice_to_form/refs/heads/main/Dataset/Healthcare%20Merged%20Dataset.csv')

In [3]:
orginal = orginal.drop('Unnamed: 3', axis=1)
orginal.head()

,ID,Patient Information,Symptoms & History
0,1,"Name: Robert M., Age: 42, Gender: Male","Doctor, I’ve been feeling dizzy whenever I exe..."
1,2,"Name: Sarah T., Age: 25, Gender: Female","My face keeps breaking out, especially around ..."
2,3,"Name: John D., Age: 50, Gender: Male",I’ve been feeling weak and fatigued throughout...
3,4,"Name: Emily S., Age: 28, Gender: Female",There’s a painful knot in my shoulder blade th...
4,5,"Name: Michael B., Age: 34, Gender: Male","When I wake up in the morning, my body feels e..."


In [4]:
len(orginal)

544

In [5]:
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('Healthcareapi')

In [6]:
from sklearn.model_selection import train_test_split


In [7]:
random_state = 42
train_size = 0.3
validation_size = 0.3
test_size = 0.4
train_df, temp_df = train_test_split(orginal, train_size=train_size, random_state=random_state)
val_df, test_df = train_test_split(temp_df, train_size=validation_size / (validation_size + test_size), random_state=random_state)


In [8]:
train_df.to_csv('part1.csv', index=False)
val_df.to_csv('part2.csv', index=False)
test_df.to_csv('part3.csv', index=False)

In [10]:
df = train_df.head(31)

In [ ]:
import google.generativeai as genai
import pandas as pd
import os

# Configure your API key
# GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY") # Ensure you have set your API key as an environment variable
genai.configure(api_key=GOOGLE_API_KEY)

model = genai.GenerativeModel('Gemini 2.0 Flash-Lite')

def extract_patient_info(text):
    """
    Extracts patient information from a text transcript using Gemini.
    """

    prompt = f"""
    Given the following patient transcript, extract the following information if available.
    If not available, return " ".

    Transcript:
    {text}

    Requested Information:
    Primary Symptoms:
    Duration of Symptoms:
    Severity (Mild, Moderate, Severe):
    Past Medical Conditions:
    Hospitalizations (reason and year):
    Allergies:
    Current Medications (name, dosage, frequency):
    Smoking (Yes/No, quantity per day):
    Alcohol Consumption (Yes/No, frequency):
    Exercise Routine (Yes/No, frequency):
    Dietary Habits (Vegetarian, Non-Vegetarian, Vegan):
    Sleep Pattern (Hours per day, quality of sleep):
    Triggering Factors:

    Output the information in a JSON-like format, where each requested information is a key and the extracted value is the corresponding value.
    """

    try:
        response = model.generate_content(prompt)
        result = response.text

        # Basic parsing to create a dictionary. Improved parsing might be needed based on the output format.
        info = {}
        lines = result.split('\n')
        for line in lines:
            if ":" in line:
                key, value = line.split(":", 1)
                info[key.strip()] = value.strip()
        return info
    except Exception as e:
        print(f"Error processing text: {e}")
        return {}

def process_dataset(df):
    """
    Processes a DataFrame with patient transcripts and extracts information.
    """
    extracted_data = []
    for index, row in df.iterrows():
        text = row["Symptoms & History"]
        extracted_info = extract_patient_info(text)
        extracted_info["ID"] = row["ID"] # Add ID for clarity
        extracted_data.append(extracted_info)

    return pd.DataFrame(extracted_data)


# Process the dataset
extracted_df = process_dataset(df)

# Print the extracted DataFrame
# print(extracted_df)

Error processing text: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.
Error processing text: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.


Error processing text: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.
Error processing text: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.


Error processing text: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.
Error processing text: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.


Error processing text: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.
Error processing text: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.


Error processing text: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.
Error processing text: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.


...

Error processing text: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.
Error processing text: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.


Error processing text: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.
Error processing text: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.


Error processing text: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.
Error processing text: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.


Error processing text: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.
Error processing text: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.


KeyboardInterrupt: 

In [13]:
import google.generativeai as genai
import pandas as pd
import os
import time

from google.colab import userdata
GOOGLE_API_KEY = userdata.get('Healthcareapi')

# Configure your API key
# GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")  # Ensure you have set your API key as an environment variable
genai.configure(api_key=GOOGLE_API_KEY)

model = genai.GenerativeModel('Gemini 2.0 Flash-Lite')

def extract_patient_info(text):
    """
    Extracts patient information from a text transcript using Gemini.
    """

    prompt = f"""
    Given the following patient transcript, extract the following information if available.
    If not available, return " ".

    Transcript:
    {text}

    Requested Information:
    Primary Symptoms:
    Duration of Symptoms:
    Severity (Mild, Moderate, Severe):
    Past Medical Conditions:
    Hospitalizations (reason and year):
    Allergies:
    Current Medications (name, dosage, frequency):
    Smoking (Yes/No, quantity per day):
    Alcohol Consumption (Yes/No, frequency):
    Exercise Routine (Yes/No, frequency):
    Dietary Habits (Vegetarian, Non-Vegetarian, Vegan):
    Sleep Pattern (Hours per day, quality of sleep):
    Triggering Factors:

    Output the information in a JSON-like format, where each requested information is a key and the extracted value is the corresponding value.
    """

    try:
        response = model.generate_content(prompt)
        result = response.text

        # Basic parsing to create a dictionary. Improved parsing might be needed based on the output format.
        info = {}
        lines = result.split('\n')
        for line in lines:
            if ":" in line:
                key, value = line.split(":", 1)
                info[key.strip()] = value.strip()
        return info
    except Exception as e:
        print(f"Error processing text: {e}")
        return {}

def process_dataset(df):
    """
    Processes a DataFrame with patient transcripts and extracts information, respecting a rate limit of 30 requests per minute.
    """
    extracted_data = []
    request_count = 0
    start_time = time.time()

    for index, row in df.iterrows():
        text = row["Symptoms & History"]
        extracted_info = extract_patient_info(text)
        extracted_info["ID"] = row["ID"]  # Add ID for clarity
        extracted_data.append(extracted_info)

        request_count += 1

        # Enforce rate limit
        if request_count >= 30:
            elapsed_time = time.time() - start_time
            if elapsed_time < 60:
                time.sleep(60 - elapsed_time)  # Wait for the remaining time
            request_count = 0
            start_time = time.time()

    return pd.DataFrame(extracted_data)

# Process the dataset
extracted_df = process_dataset(df)


Error processing text: 400 POST https://generativelanguage.googleapis.com/v1beta/models/Gemini%202.0%20Flash-Lite:generateContent?%24alt=json%3Benum-encoding%3Dint: * GenerateContentRequest.model: unexpected model name format

Error processing text: 400 POST https://generativelanguage.googleapis.com/v1beta/models/Gemini%202.0%20Flash-Lite:generateContent?%24alt=json%3Benum-encoding%3Dint: * GenerateContentRequest.model: unexpected model name format



Error processing text: 400 POST https://generativelanguage.googleapis.com/v1beta/models/Gemini%202.0%20Flash-Lite:generateContent?%24alt=json%3Benum-encoding%3Dint: * GenerateContentRequest.model: unexpected model name format

Error processing text: 400 POST https://generativelanguage.googleapis.com/v1beta/models/Gemini%202.0%20Flash-Lite:generateContent?%24alt=json%3Benum-encoding%3Dint: * GenerateContentRequest.model: unexpected model name format



Error processing text: 400 POST https://generativelanguage.googleapis.com/v1beta/models/Gemini%202.0%20Flash-Lite:generateContent?%24alt=json%3Benum-encoding%3Dint: * GenerateContentRequest.model: unexpected model name format

Error processing text: 400 POST https://generativelanguage.googleapis.com/v1beta/models/Gemini%202.0%20Flash-Lite:generateContent?%24alt=json%3Benum-encoding%3Dint: * GenerateContentRequest.model: unexpected model name format



Error processing text: 400 POST https://generativelanguage.googleapis.com/v1beta/models/Gemini%202.0%20Flash-Lite:generateContent?%24alt=json%3Benum-encoding%3Dint: * GenerateContentRequest.model: unexpected model name format

Error processing text: 400 POST https://generativelanguage.googleapis.com/v1beta/models/Gemini%202.0%20Flash-Lite:generateContent?%24alt=json%3Benum-encoding%3Dint: * GenerateContentRequest.model: unexpected model name format



Error processing text: 400 POST https://generativelanguage.googleapis.com/v1beta/models/Gemini%202.0%20Flash-Lite:generateContent?%24alt=json%3Benum-encoding%3Dint: * GenerateContentRequest.model: unexpected model name format

Error processing text: 400 POST https://generativelanguage.googleapis.com/v1beta/models/Gemini%202.0%20Flash-Lite:generateContent?%24alt=json%3Benum-encoding%3Dint: * GenerateContentRequest.model: unexpected model name format



...

Error processing text: 400 POST https://generativelanguage.googleapis.com/v1beta/models/Gemini%202.0%20Flash-Lite:generateContent?%24alt=json%3Benum-encoding%3Dint: * GenerateContentRequest.model: unexpected model name format

Error processing text: 400 POST https://generativelanguage.googleapis.com/v1beta/models/Gemini%202.0%20Flash-Lite:generateContent?%24alt=json%3Benum-encoding%3Dint: * GenerateContentRequest.model: unexpected model name format



Error processing text: 400 POST https://generativelanguage.googleapis.com/v1beta/models/Gemini%202.0%20Flash-Lite:generateContent?%24alt=json%3Benum-encoding%3Dint: * GenerateContentRequest.model: unexpected model name format



Error processing text: 400 POST https://generativelanguage.googleapis.com/v1beta/models/Gemini%202.0%20Flash-Lite:generateContent?%24alt=json%3Benum-encoding%3Dint: * GenerateContentRequest.model: unexpected model name format

Error processing text: 400 POST https://generativelanguage.googleapis.com/v1beta/models/Gemini%202.0%20Flash-Lite:generateContent?%24alt=json%3Benum-encoding%3Dint: * GenerateContentRequest.model: unexpected model name format



Error processing text: 400 POST https://generativelanguage.googleapis.com/v1beta/models/Gemini%202.0%20Flash-Lite:generateContent?%24alt=json%3Benum-encoding%3Dint: * GenerateContentRequest.model: unexpected model name format

Error processing text: 400 POST https://generativelanguage.googleapis.com/v1beta/models/Gemini%202.0%20Flash-Lite:generateContent?%24alt=json%3Benum-encoding%3Dint: * GenerateContentRequest.model: unexpected model name format



KeyboardInterrupt: 

In [ ]:
extracted_df